In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
#import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import tensorflow as tf
directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

In [2]:
dev_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/dev.users'
dev_users_list = []
with open(dev_users_path, 'r') as fr:
    lines = fr.readlines()
    dev_users_list = lines
    del lines
for i in range(len(dev_users_list)):
    dev_users_list[i] = dev_users_list[i].replace('\n','')
    
test_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/test.users'
test_users_list = []
with open(test_users_path, 'r') as fr:
    lines = fr.readlines()
    test_users_list = lines
    del lines
for i in range(len(test_users_list)):
    test_users_list[i] = test_users_list[i].replace('\n','')

print(len(dev_users_list))
print(dev_users_list[:5])
print(len(test_users_list))
print(test_users_list[:5])    

3000
['#d6866a498157771069fdf15361cb012b', '#f963fb8c5d9d14d503fc4e80bd8617b4', '#87a6479c91e4276374378f1d28eb307c', '#677e984e245b344f61dc5d3cc1f352c8', '#519f45eb14e4807e8714fb7e835463eb']
5000
['#7ee14df8642a7925b1465ff5c89efe5b', '#8420b9385b282028eebf1ad6b4a221c0', '#c9b31d8b64357f5854b1ba55b32eb6d3', '#9bb1e13b5481fa3737af20870b25c723', '#37d5f99a7f12c9ba90c4e2ac92e54ab6']


# Load Article list

In [3]:
with open(directory+'article_by_user.json', 'rb') as f:
    article_by_nontestuser = json.load(f)

with open(directory+'article_by_testuser.json', 'rb') as f:
    article_by_testuser = json.load(f)

In [4]:
print(len(article_by_nontestuser))
print(len(article_by_testuser))

301222
5000


## article list processing 
- 최신 글부터 중복된 글을 제거하고 다시 dictionary를 만듦 (최신 글이 중요하다는 생각으로)

In [6]:
import time

In [7]:
start_time = time.time()
article_by_nontestuser_t = dict()
for key,value in article_by_nontestuser.items() :
    key = int(key)
    s = []
    value = list(reversed(value))
    for i in range(len(value)):
        if value[i] not in s:
            s.append(value[i])
    s = list(reversed(s))
    article_by_nontestuser_t[key] = s
print(time.time() - start_time , 'sec')

85.53485441207886 sec


In [8]:
print(len(article_by_nontestuser_t))

301222


In [9]:
start_time = time.time()
article_by_testuser_t = dict()
for key,value in article_by_testuser.items() :
    key = int(key)
    s = []
    value = list(reversed(value))
    for i in range(len(value)):
        if value[i] not in s:
            s.append(value[i])
    s = list(reversed(s))
    article_by_testuser_t[key] = s
    
print(time.time() - start_time , 'sec')

19.241394519805908 sec


In [10]:
print(len(article_by_testuser_t))

5000


In [ ]:
# Article 중복 제거 해보기 (역순으로)

# Read Procesing Again

- test_users 가 읽은 글들을 다시 뽑기 (5000개의 글) - 제대로 됐음

- article_by_nontestuser_t
- article_by_testuser_t

In [11]:
import time
from tqdm import tqdm

# [1] Make Dict

In [12]:
start_time = time.time()
read_file_lst = glob.glob(directory + 'read/read/*')
exclude_file_lst = ['read.tar']
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        read_df_lst.append(df_temp)
        
read = pd.concat(read_df_lst)
print ("%s sec"%(time.time() - start_time))

50.146909952163696 sec


In [13]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))
read_cnt_by_user = read['article_id'].str.split(' ').map(len)
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})
read_raw.shape



(22110706, 4)

In [14]:
metadata = pd.read_json(directory + 'metadata.json', lines=True)

In [15]:
atc = metadata.copy()
atc['reg_datetime'] = atc['reg_ts'].apply(lambda x : datetime.fromtimestamp(x/1000.0))
atc.loc[atc['reg_datetime'] == atc['reg_datetime'].min(), 'reg_datetime'] = datetime(2090, 12, 31)
atc['reg_dt'] = atc['reg_datetime'].dt.date
atc['type'] = atc['magazine_id'].apply(lambda x : '개인' if x == 0.0 else '매거진')
# 컬럼명 변경
atc.columns = ['id', 'display_url', 'article_id', 'keyword_list', 'magazine_id', 'reg_ts', 'sub_title', 'title', 'author_id', 'reg_datetime', 'reg_dt', 'type']
atc.head()
atc_cnt_by_reg_dt = atc.groupby('reg_dt', as_index=False)['article_id'].count()

In [16]:
atc_read_cnt = read_raw[read_raw.article_id != ''].groupby('article_id')['user_id'].count()
atc_read_cnt = atc_read_cnt.reset_index()
atc_read_cnt.columns = ['article_id', 'read_cnt']
#metadata 결합
atc_read_cnt = pd.merge(atc_read_cnt, atc, how='left', left_on='article_id', right_on='article_id')
# metadata를 찾을 수 없는 소비 로그 제외
atc_read_cnt_nn = atc_read_cnt[atc_read_cnt['id'].notnull()]
# 소비수 기준 분류값
#atc_read_cnt_nn.sort_values(by='read_cnt', ascending=False).head(2)

### len(20) 미만 버리기

In [17]:
import time
start_time = time.time()
atc_read_cnt = atc_read_cnt_nn[atc_read_cnt_nn['read_cnt'] > 20]
article_vocab_list = atc_read_cnt['article_id'].tolist()
print(len(article_vocab_list))
print ("%s sec"%(time.time() - start_time))

134991
0.14211320877075195 sec


In [18]:
import time
start_time = time.time()
atc_read_cnt_less = atc_read_cnt_nn[atc_read_cnt_nn['read_cnt'] <= 20]
article_vocab_list_less = atc_read_cnt_less['article_id'].tolist()
print(len(article_vocab_list_less))
print ("%s sec"%(time.time() - start_time))

341156
0.14711737632751465 sec


In [ ]:
- article_by_nontestuser_t
- article_by_testuser_t

In [19]:
article_vocab_list_nontestuser_t = set()
for values in article_by_nontestuser_t.values():
    for x in values:
        article_vocab_list_nontestuser_t.add(x)

In [20]:
article_vocab_list_testuser_t = set()
for values in article_by_testuser_t.values():
    for x in values:
        article_vocab_list_testuser_t.add(x)

In [21]:
article_vocab_list_byuser = article_vocab_list_nontestuser_t | article_vocab_list_testuser_t

In [22]:
print(len(article_vocab_list))
print(len(article_vocab_list_byuser))
print(len(article_vocab_list_less))

134991
505840
341156


In [23]:
article_vocab_list_byuser = list(set(article_vocab_list_byuser) - set(article_vocab_list))
article_vocab_list_less = list(set(article_vocab_list_less) - set(article_vocab_list))

In [24]:
article_vocab_list_byuser = list(set(article_vocab_list_byuser) - set(article_vocab_list_less))

In [25]:
print(len(article_vocab_list))
print(len(article_vocab_list_byuser))
print(len(article_vocab_list_less))

134991
29693
341156


In [26]:
nonpopular_article_vocab_list = article_vocab_list_less + article_vocab_list_byuser
print(len(nonpopular_article_vocab_list))

370849


### Dict 만들기

In [27]:
# make dictionary
# generate dataset, dictionary for word
start_time = time.time()
word_to_id = {}
id_to_word = {}

for word in article_vocab_list:
    new_id = len(word_to_id)
    word_to_id[word] = new_id
    id_to_word[new_id] = word

# 공백 표시할 어휘 : eos
word_to_id['eos'] = len(word_to_id)
id_to_word[len(word_to_id)-1] = 'eos'
# vocab에 없는 단어는 UNK로 표시
word_to_id['UNK'] = len(word_to_id)
id_to_word[len(word_to_id)-1] = 'UNK'

print ("%s sec"%(time.time() - start_time))

0.09408259391784668 sec


In [28]:
print(len(word_to_id))
print(len(id_to_word))

134993
134993


In [29]:
fix_id = len(word_to_id)

In [30]:
fix_id

134993

In [31]:
for word in nonpopular_article_vocab_list:
    word_to_id[word] = fix_id
    id_to_word[fix_id] = 'UNK'

In [32]:
print(len(word_to_id))
print(len(id_to_word))

505842
134994


In [33]:
with open(directory + 'word_to_id.json','w') as f:
    json.dump(word_to_id,f)
    
with open(directory + 'id_to_word.json','w') as f:
    json.dump(id_to_word,f)

with open(directory + 'word_to_id.json') as f:
    word_to_id  = json.load(f)

with open(directory + 'id_to_word.json') as f:
    id_to_word = json.load(f)

### [2] make corpus

In [34]:
total_article_list_by_users = list(article_by_nontestuser_t.values()) + list(article_by_testuser_t.values())
testuser_article_list_by_users = list(article_by_testuser_t.values())

In [35]:
import random
random.shuffle(total_article_list_by_users)

In [36]:
start_time = time.time()
corpus = []

  
for values in total_article_list_by_users:

    if(len(values) > 4):
        for x in values:
            corpus.append(x)
    
    if(len(values) > 4):
        for i in range(8):
            corpus.append('eos')
        
        
print ("%s sec"%(time.time() - start_time))
print(len(corpus))

1.8805170059204102 sec
13907947


In [37]:
start_time = time.time()
corpus_r = []
corpus_r = [word_to_id[w] for w in corpus]
print ("%s sec"%(time.time() - start_time))
print(len(corpus_r))

4.974085330963135 sec
13907947


### [4] Model 

- recommend 시 필요없음

In [38]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import collections
def generate_batch(data, batch_size, num_skips, skip_window, data_index):
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window

    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    span = 2 * skip_window + 1                      # context = skip_window + target + skip_window
    assert span > num_skips

    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)   # 다음 단어 인덱스로 이동. len(data) = 17005207

    for i in range(batch_size // num_skips):

        targets = list(range(span))     # 1. 0부터 span-1까지의 정수로 채운 다음
        targets.pop(skip_window)        # 2. skip_window번째 삭제
        np.random.shuffle(targets)      # 3. 난수를 사용해서 섞는다.

        start = i * num_skips
        batch[start:start+num_skips] = buffer[skip_window]

        for j in range(num_skips):
            labels[start+j, 0] = buffer[targets[j]]

        # 새로운 요소가 들어가면서 가장 먼저 들어간 데이터 삭제
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels, data_index

In [39]:
# Step 4: skip-gram 모델 구축
import math
np.random.seed(1)
tf.set_random_seed(1)
vocabulary_size =(len(id_to_word)) ################################################# 수정해주기!!
batch_size = 128        # 일반적으로 16 <= batch_size <= 512
embedding_size = 128    # embedding vector 크기
skip_window = 4         # target 양쪽의 단어 갯수
num_skips = 8          # 컨텍스트로부터 생성할 레이블 갯수

valid_size = 5     # 유사성을 평가할 단어 집합 크기
valid_window = 15000  # 앞쪽에 있는 분포들만 뽑기 위한 샘플
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # negative 샘플링 갯수

train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

truncated = tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size))
nce_weights = tf.Variable(truncated)
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# embeddings 벡터.
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

# 배치 데이터에 대해 NCE loss 평균 계산
nce_loss = tf.nn.nce_loss(weights=nce_weights,
                          biases=nce_biases,
                          labels=train_labels,
                          inputs=embed,
                          num_sampled=num_sampled,
                          num_classes=vocabulary_size)
loss = tf.reduce_mean(nce_loss)

# SGD optimizer
optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

# 유사도를 계산하기 위한 모델. 학습 모델은 optimizer까지 구축한 걸로 종료.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

W0721 17:03:08.941899 22088 deprecation.py:323] From D:\ANACONDA\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0721 17:03:09.069992 22088 deprecation.py:506] From <ipython-input-39-2fea1435b5f3>:42: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [40]:
# Step 5: skip-gram 모델 학습
# 4시 4분 시작
start_time = time.time()
num_steps = 1000001
data = corpus_r #################################### 수정
ordered_words = list(word_to_id.keys())
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    average_loss, data_index = 0, 0
    for step in range(num_steps):
        batch_inputs, batch_labels, data_index = generate_batch(data, batch_size, num_skips, skip_window, data_index)

        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        # 마지막 10000번에 대한 평균 loss 표시
        if step % 10000 == 0:
            if step > 0:
                average_loss /= 10000
            print('Average loss at step {} : {}'.format(step, average_loss))
            average_loss = 0
        
        # 10만번째마다 valid size만큼 sim 계산
        if step % 100000 == 0:
            sim = similarity.eval()         # (16, vocab_size)

            for i in range(valid_size):
                valid_word = ordered_words[valid_examples[i]]

                top_k = 8
                nearest = sim[i].argsort()[-top_k - 1:-1][::-1]
                log_str = ', '.join([ordered_words[k] for k in nearest])
                print('Nearest to {}: {}'.format(valid_word, log_str))

    final_embeddings = normalized_embeddings.eval()

print(time.time() - start_time , 'sec')

Average loss at step 0 : 334.28741455078125
Nearest to @artinsight_399: @jaol_1, @shoong810_192, @jumjan_257, @plusx_26, @vpluscharles_42, @kecologist_70, @hopeingyu_5, @kyleso_7
Nearest to @bestar_31: @barneconomy_79, @readingboy_43, @jawssss_13, @videsign20_16, @likewater_23, @brunch5nga_3, @munchi_65, @babydiary_146
Nearest to @alexkang_1012: @iamyoungzoo_17, @myungsunkim_269, @syn701_75, @leeojsh_22, @ljg523_1753, @tenbody_1649, @sustainability_31, @lookfar_27
Nearest to @altctrlshift_35: @lookfar_164, @20timeline_33, @brunchqxk5_181, @allstay_1044, @yunj814_94, @leedh200_10, @ujinnew_7, @brunch1uhl_567
Nearest to @13july_65: @needleworm_10, @gians_13, @everj76_161, @mokiusagi_12, @failit_28, @znryu_38, @beaurin58_5, @malcom_46
Average loss at step 10000 : 113.89247313814163
Average loss at step 20000 : 46.4074169931364
Average loss at step 30000 : 29.108286816103348
Average loss at step 40000 : 19.39349226460999
Average loss at step 50000 : 14.566657371038467
Average loss at step 

In [42]:
np.save(directory+'article_embedding_matrix_t',final_embeddings)

# Similarity

In [2]:
final_embeddings = np.load(directory+'article_embedding_matrix_t.npy')

In [5]:
import time
from tqdm import tqdm

In [43]:
def cos_matrix_multiplication(matrix, vector):
    """
    Calculating pairwise cosine distance using matrix vector multiplication.
    """
    dotted = matrix.dot(vector)
    matrix_norms = np.linalg.norm(matrix, axis=1)
    vector_norm = np.linalg.norm(vector)
    matrix_vector_norms = np.multiply(matrix_norms, vector_norm)
    neighbors = np.divide(dotted, matrix_vector_norms)
    return neighbors

In [44]:
start_time = time.time()
cos_matrix_multiplication(final_embeddings, final_embeddings[0])

print ("%s sec"%(time.time() - start_time))

0.04402565956115723 sec


In [45]:
with open(directory + 'word_to_id.json') as f:
    word_to_id  = json.load(f)

with open(directory + 'id_to_word.json') as f:
    id_to_word = json.load(f)

In [46]:
print(len(word_to_id))
print(len(id_to_word))

505842
134994


In [47]:
from gensim.models import KeyedVectors, Word2Vec
from keras.preprocessing.text import Tokenizer

st_t = time.time()
tokenizer = Tokenizer()
f = open(directory+'gensim.txt' ,'w', encoding='utf8')
f.write(f'{len(id_to_word)} {128}\n')

word_vecs = final_embeddings  # W Matrix

for idx, word in id_to_word.items():
    str_vec = ' '.join(map(str, list(word_vecs[int(idx), :])))
    f.write(f'{word} {str_vec}\n')

f.close()
w2v_cbow = KeyedVectors.load_word2vec_format(directory+'gensim.txt', unicode_errors='ignore')
print(time.time() - st_t,'sec')

Using TensorFlow backend.
W0721 17:39:03.793826 22088 utils_any2vec.py:354] duplicate word 'UNK' in D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/gensim.txt, ignoring all but first


31.5535569190979 sec


In [48]:
s = time.time()
w2v_cbow.most_similar(id_to_word['0'])
print(time.time() -s)

0.11109757423400879


In [21]:
li = w2v_cbow.most_similar(id_to_word['0'])

In [49]:
wl = []
for i in tqdm(range(len(id_to_word)),mininterval = 1):
    li = []
    li = w2v_cbow.most_similar(id_to_word[str(i)])
    li_1 = []
    for j in range(10):
        li_1.append(li[j][0])
    wl.append(li_1)

100%|█████████████████████████████████████████████████████████████████████████| 134994/134994 [10:36<00:00, 212.18it/s]


In [52]:

wl =  np.asarray(wl)
print(wl.shape)

(134994, 10)


In [50]:
np.save(directory+'article_similarity_t',wl)

In [33]:
article_similarity = np.load(directory+'article_similarity_t.npy')

In [35]:
article_similarity[word_to_id['@yuhaaustin_45']]

array(['@yuhaaustin_52', '@yuhaaustin_54', '@yuhaaustin_10',
       '@spring0915_57', '@hukho_235', '@leesoltoon_33', '@deckey1985_51',
       '@anetmom_8', '@ohmygod_39', '@sunnysohn_60'], dtype='<U21')